In [1]:
import tensorflow as tf
import numpy as np

In [2]:
xy = np.loadtxt('07train.txt', unpack=True)
x_data = np.transpose(xy[0:-1])
y_data = np.reshape(xy[-1], (4, 1))

In [3]:
X = tf.placeholder(tf.float32, name='X-input')
Y = tf.placeholder(tf.float32, name='Y-input')

In [4]:
W1 = tf.Variable(tf.random_uniform([2,2],-1.0,1.0), name="Weight1")
W2 = tf.Variable(tf.random_uniform([2,1],-1.0,1.0), name="Weight2")
b1 = tf.Variable(tf.zeros([2]),name="Bias1")
b2 = tf.Variable(tf.zeros([1]),name="Bias2")

In [5]:
with tf.name_scope("layer2") as scope:
    L2 = tf.sigmoid(tf.matmul(X,W1)+b1)

with tf.name_scope("layer3") as scope:
    hypothesis = tf.sigmoid(tf.matmul(L2,W2)+b2)

with tf.name_scope('cost') as scope:
    cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1-Y) * tf.log(1 - hypothesis))
    cost_summ = tf.summary.scalar("cost", cost)

with tf.name_scope('train') as scope:
    a = tf.Variable(0.1)
    optimizer = tf.train.GradientDescentOptimizer(a)
    train = optimizer.minimize(cost)

In [7]:
W1_hist = tf.summary.histogram("Weights1", W1)
W2_hist = tf.summary.histogram("Weights2", W2)

b1_hist = tf.summary.histogram("biases1", b1)
b2_hist = tf.summary.histogram("biases2", b2)

y_hist = tf.summary.histogram("y", Y)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
   # tf.train.write_graph(sess.graph, './logs/xor_logs', 'graph.pbtxt')
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter("./logs/xor_logs", sess.graph)

    for step in range(20000):
        sess.run(train, feed_dict={X: x_data, Y: y_data})
        if step % 2000 == 0:
            summary = sess.run(merged, feed_dict={X: x_data, Y: y_data})
            writer.add_summary(summary, step)
            print(step, sess.run(cost, feed_dict={X: x_data, Y: y_data}), sess.run(W1), sess.run(W2))
    
    writer.add_graph(sess.graph)
    correct_prediction = tf.equal(tf.floor(hypothesis+0.5), Y)

    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print(sess.run([hypothesis, tf.floor(hypothesis+0.5), correct_prediction], feed_dict={X: x_data, Y: y_data}))
    print("accuracy", accuracy.eval({X: x_data, Y: y_data}))

0 0.70071 [[-0.03744227 -0.60465336]
 [-0.07429841  0.21340059]] [[-0.78472954]
 [ 0.28232166]]
2000 0.692819 [[-0.16114879 -0.70349944]
 [-0.05581751  0.28314903]] [[-0.71390462]
 [ 0.38347277]]
4000 0.68964 [[-0.31010604 -1.08736289]
 [-0.0412275   0.60157835]] [[-0.77334076]
 [ 0.67711258]]
6000 0.488089 [[-1.32736635 -3.72666049]
 [ 1.27818322  3.2456882 ]] [[-1.94344664]
 [ 3.61104465]]
8000 0.0682741 [[-4.46936035 -5.44504166]
 [ 4.7478075   5.25654888]] [[-6.57051039]
 [ 7.24771786]]
10000 0.0297349 [[-5.24577522 -5.94748497]
 [ 5.52329683  5.77215481]] [[-8.16605568]
 [ 8.70987892]]
12000 0.0186235 [[-5.62398005 -6.21721649]
 [ 5.89982605  6.03885841]] [[-9.04798794]
 [ 9.55832481]]
14000 0.0134736 [[-5.86655474 -6.39788628]
 [ 6.14115906  6.21534491]] [[ -9.65560246]
 [ 10.15275383]]
16000 0.0105258 [[-6.04249382 -6.53235674]
 [ 6.31619883  6.34597826]] [[-10.11862755]
 [ 10.60941219]]
18000 0.00862353 [[-6.17933416 -6.63881063]
 [ 6.45237207  6.44908381]] [[-10.49248791]
 [ 1